### imports and preparation

- place this notebook in ../typeform 
- store your own token for tf API in a file called "typeform_token.py" and save it locally (do not commit) in /typeform
- The file contains only one line: TOKEN="your_token"

In [1]:
import requests
import pandas as pd
#import my own function
%run one_entry_to_dictionary.py
%run typeform_token.py

In [16]:
import sys
sys.path.append('../')

# 1. Pull Data from Typeform

### Connect to TF, see metainfo for form

In [3]:
#IMPORT TOKEN FILE 
auth_token= TOKEN
head = {'Authorization': 'Bearer ' + auth_token}
url = 'https://api.typeform.com/forms/OHziDn'

metainfo_form = requests.get(url, headers=head)
#response 200 means its good !
print(metainfo_form)
print(metainfo_form.json())

<Response [200]>
{'id': 'OHziDn', 'title': 'UN Women COVID-19 Survey', 'theme': {'href': 'https://api.typeform.com/themes/UTPtTC'}, 'workspace': {'href': 'https://api.typeform.com/workspaces/y2fgNQ'}, 'settings': {'is_public': True, 'is_trial': False, 'language': 'en', 'progress_bar': 'percentage', 'show_progress_bar': False, 'show_typeform_branding': False, 'meta': {'allow_indexing': False, 'description': "Help us understand women's experience during COVID-19.", 'image': {'href': 'https://images.typeform.com/images/r8wQiuyGeEu2'}}}, 'welcome_screens': [{'ref': '427323db-ebf3-48b6-8cf0-2f2ecdcebd7f', 'title': "Tell us how you're doing during lockdown with a few short questions, so we can make sure women's voices are heard.\n\nWe'll use your answers to show the government and leaders what changes need to happen.\n\nThank you so much for being part of this.", 'properties': {'show_button': True, 'button_text': 'Start'}, 'attachment': {'type': 'image', 'href': 'https://images.typeform.com/

### Pull all responses exclusing mail from Typeform

In [4]:
# get all answers but not emails
SIZE = "500"
url = "https://api.typeform.com/forms/OHziDn/responses?fields=Xa2T0bcZ5VnY,QUtA6k0gcrZf,ZNZyNVwMtwID,azlKRkvdcc7O,Vrzj2uv0i29C,Xh2W1d6FhXyK,t9EaTsONnJfR,OaujoVSY6NaS,ASolp4v3adZK,erHBE6gKxycf,ASQmq4h3vxRt,KEGztC8iYfyy,kWb7LUjdJyhb,g9goY6hIxmC9,jG2wzvAmOWxq,HNqA7BK7ZDFl&page_size="+SIZE

response = requests.get(url, headers=head)
#response 200 means its good !
print(response)

responses = response.json()

<Response [200]>


In [5]:
print("number of responses",responses['total_items'])
#look at the first response
print("the last response:",responses["items"][0])

number of responses 283
the last response: {'landing_id': '2jq17ikprbr4l0xxhrl2jq17f8f20zo5', 'token': '2jq17ikprbr4l0xxhrl2jq17f8f20zo5', 'response_id': '2jq17ikprbr4l0xxhrl2jq17f8f20zo5', 'landed_at': '2020-05-12T09:57:01Z', 'submitted_at': '2020-05-12T10:02:49Z', 'metadata': {'user_agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 13_4_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Mobile/15E148 Safari/604.1', 'platform': 'mobile', 'referer': 'https://unwomenuk.typeform.com/', 'network_id': '3fcb8239d6', 'browser': 'touch'}, 'calculated': {'score': 0}, 'answers': [{'field': {'id': 'Xa2T0bcZ5VnY', 'ref': '49c4acab-fed2-46c2-804d-8f36db71a6d5', 'type': 'multiple_choice'}, 'type': 'choice', 'choice': {'label': 'Woman'}}, {'field': {'id': 'QUtA6k0gcrZf', 'ref': '81c603ce-bdf7-4eb1-b7ac-6c489e514d80', 'type': 'multiple_choice'}, 'type': 'choice', 'choice': {'label': '25-34'}}, {'field': {'id': 'ZNZyNVwMtwID', 'ref': 'e9a85e52-f699-49da-af49-691eb1bcb8c8', 'type': 'multi

# 2. Transform JSON to pandas.df and map dummy vars

### step by step

In [6]:
# LOAD ONE PERSON FROM THE JSON RESPONSE
one_person = responses["items"][0]

In [7]:
#transform ONE person's information to a dictionary using function one_entry_to_dictionary
dictionary_test = one_entry_to_dictionary(one_person)
print(dictionary_test)

{'id': '2jq17ikprbr4l0xxhrl2jq17f8f20zo5', 'submitted_at': '2020-05-12T10:02:49Z', 'gender': 'Woman', 'age': '25-34', 'ethnicity': 'Black/African Caribbean/Black British', 'live_alone': 0, 'live_friends': 0, 'live_partner': 1, 'live_parents': 0, 'live_siblings': 0, 'live_otherfamily': 0, 'live_children': 1, 'live_other': 0, 'location': 'Sm2 6dy', 'safety_level': 1, 'safety_change': 'Worse', 'safety_why': 'I have noticed a increase of illegal activities happening on the streets ie drugs. I have also been a victim of identity fraud.', 'mental_scale': 3, 'mental_problems_repetitive': 1, 'mental_problems_lonely': 1, 'mental_problems_stressed': 1, 'mental_problems_anxious': 1, 'mental_problems_worriedhealth': 0, 'mental_problems_scaredfuture': 1, 'mental_problems_relationship': 0, 'mental_problems_other': 0, 'mental_problems_none': 0, 'positive_impact_support_housework': 0, 'positive_impact_support_childcare': 0, 'positive_impact_connect': 0, 'positive_impact_leisure': 0, 'positive_impact_w

In [30]:
# transform dictionary to df 
df_test = pd.DataFrame(dictionary_test,index=[0])
df_test.head()

,id,submitted_at,gender,age,ethnicity,live_alone,live_friends,live_partner,live_parents,live_siblings,...,positive_impact_connect,positive_impact_leisure,positive_impact_wfh,positive_impact_other,positive_impact_none,work_situation,financial_situation,housework_amount,housework_change,testimonial
0,2jq17ikprbr4l0xxhrl2jq17f8f20zo5,2020-05-12T10:02:49Z,Woman,25-34,Black/African Caribbean/Black British,0,0,1,0,0,...,0,0,0,0,0,"Yes, I work at home",It has had a bit of a negative impact and has ...,I take care of all the housework,I do more and it's impacting my paid work,Uncertainty in every area


### apply this to entire JSON

In [32]:
def transform_all_to_df(responses):
    '''
    transforms the JSON response from the above query into a df with cols=vars, rows=individuals
    '''
    no_df_defined = True
    for item in responses["items"]:
        dictionary_transformation = one_entry_to_dictionary(item)
        df = pd.DataFrame(dictionary_transformation,index=[0])
        #crate dictionary and then append to it 
        if no_df_defined:
            df_all = df
            no_df_defined = False
        else: 
            df_all = df_all.append(df)
        df_all = df_all.reset_index(drop = True)
    return df_all

In [33]:
df_all = transform_all_to_df(responses)

j3how3ucddd42l75uj3how3unt6vj27w has entered in column other_work: In between jobs // THIS ENTRY WAS DISCARDED AND SAVED AS OTHER
8lg2mtsg3ala9xkk8lg2mdirml0bf04a has entered in column other_work: Maternity Leave // THIS ENTRY WAS DISCARDED AND SAVED AS OTHER
g3re0wvwmge256f6g3re3c09cv3hn7ej has entered in column other_work: Maternity  // THIS ENTRY WAS DISCARDED AND SAVED AS OTHER
yhqphqlb1d8tj14yhqphqege43pe5f0w has entered in column other_work: I work in Internet as hobby, I don't get money for that yet. // THIS ENTRY WAS DISCARDED AND SAVED AS OTHER
w577oearvygk25d1w577zi41rxwtd5x7 has entered in column other_work: Stay at home mum  // THIS ENTRY WAS DISCARDED AND SAVED AS OTHER
gym89ky1wt2f7urmmo9nsgym89kyffbg has entered in column other_work: Stay at home with kids // THIS ENTRY WAS DISCARDED AND SAVED AS OTHER


In [34]:
df_all.head()

,id,submitted_at,gender,age,ethnicity,live_alone,live_friends,live_partner,live_parents,live_siblings,...,positive_impact_connect,positive_impact_leisure,positive_impact_wfh,positive_impact_other,positive_impact_none,work_situation,financial_situation,housework_amount,housework_change,testimonial
0,2jq17ikprbr4l0xxhrl2jq17f8f20zo5,2020-05-12T10:02:49Z,Woman,25-34,Black/African Caribbean/Black British,0,0,1,0,0,...,0,0,0,0,0,"Yes, I work at home",It has had a bit of a negative impact and has ...,I take care of all the housework,I do more and it's impacting my paid work,Uncertainty in every area
1,2ne9wmkxwrmbaav8cl6bc2ne9wk1zsgd,2020-05-12T09:43:09Z,Woman,45-54,Black/African Caribbean/Black British,0,0,0,0,0,...,0,0,0,0,0,"Yes, I work at home",I feel better about my finances than I did before,I take care of most of the housework,I now do less,Enjoying time with my children - however worri...
2,hx10ke8r8voeonj2xg3j8tmhx10k8hvz,2020-05-12T09:23:59Z,Woman,25-34,White,0,0,1,0,0,...,0,0,0,0,0,"Yes, I work at home",It has had a bit of a negative impact and has ...,I take care of most of the housework,I haven't noticed a change,NA
3,0bzjymt7iz2qlwfgs0bz1ca7npbd0ld1,2020-05-12T09:01:42Z,Woman,25-34,White,0,0,1,0,0,...,0,0,0,0,0,"Yes, I work outside of my home",It has had a bit of a negative impact and has ...,I take care of all the housework,I haven't noticed a change,0
4,1h4cywh5rrt1lmj3q1h4r056kcje82gq,2020-05-12T08:37:08Z,Woman,25-34,White,0,0,1,0,0,...,0,0,0,0,0,"Yes, I work at home",I feel better about my finances than I did before,I take care of most of the housework,I do more but it's manageable and sustainable,Of course it has been a totally unexpected and...


In [35]:
len(df_all)

283

In [36]:
df_all.columns

Index(['id', 'submitted_at', 'gender', 'age', 'ethnicity', 'live_alone',
       'live_friends', 'live_partner', 'live_parents', 'live_siblings',
       'live_otherfamily', 'live_children', 'live_other', 'location',
       'safety_level', 'safety_change', 'safety_why', 'mental_scale',
       'mental_problems_repetitive', 'mental_problems_lonely',
       'mental_problems_stressed', 'mental_problems_anxious',
       'mental_problems_worriedhealth', 'mental_problems_scaredfuture',
       'mental_problems_relationship', 'mental_problems_other',
       'mental_problems_none', 'positive_impact_support_housework',
       'positive_impact_support_childcare', 'positive_impact_connect',
       'positive_impact_leisure', 'positive_impact_wfh',
       'positive_impact_other', 'positive_impact_none', 'work_situation',
       'financial_situation', 'housework_amount', 'housework_change',
       'testimonial'],
      dtype='object')

# Aggregation for Graph 0 - Map 

In [76]:
%load_ext autoreload
%autoreload 2
from overview_tab.map_graph import merge_local_internat_dataframe

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
postal_code = pd.read_csv("https://raw.githubusercontent.com/rjcnrd/domestic_violence_covid-19/master/data/postcode_uk.csv", index_col = 0)
#countries = pd.read_csv("https://raw.githubusercontent.com/rjcnrd/domestic_violence_covid-19/master/data/countries.csv")
countries = pd.read_csv("/Users/ameliemeurer/Documents/Amélie/HEC M2/07. Research Paper - UN Women/02. Code/domestic_violence_covid-19/data/countries.csv", index_col = 0)

In [78]:
all_reports_df, safety_df, safety_change_df, mental_health_df, working_situation_df, not_in_map, uk_data, uk_data_county = merge_local_internat_dataframe(survey_df = df_all, postal_code_df = postal_code, countries_df = countries,  map_threshold = 0, postal_code_col = "location")

In [ ]:
all_reports_df.to_csv("data_map_all.csv")
safety_df.to_csv("data_map_safety_scale.csv")
safety_change_df.to_csv("data_map_safety_change.csv")
mental_health_df.to_csv("data_map_mental_health.csv")
working_situation_df.to_csv("data_map_working_situation.csv")

## Check for the rows that are not integrated

In [79]:
#Here are the values that are not integrated, by order
not_in_map.location.value_counts()

LONDON                      7
UK                          2
ENGLAND                     2
UNITED STATES OF AMERICA    1
HONG KONG SAR               1
KEGWORTH                    1
01-248 POLAND               1
ΝR2 2DR                     1
TOOTING                     1
PARIS                       1
700019 INDIA                1
7 0RB                       1
WOLVERHAMPTON UK            1
BERLIN GERMANY              1
UNITED KINGDOM              1
CHESTERFIELD                1
HKSAR                       1
HOVE                        1
Name: location, dtype: int64

In [ ]:
#If you want to append new location, you should update the countries csv, one by one 
#country_name : name of the country as you see it in the survey
#Area : name of the country as it appears already in the survey. THIS WILL BE USED FOR A MERGE
#Area_name : name of the country as it should appear in the hover (the same name as area name)
"""
countries_new = countries.append({"latitude": 22.396428, 
                                  "longitude": 114.10949699999999, 
                                  "country_name" : "HKSAR",
                                  "area": "HONG KONG", 
                                  "area_name": "Hong Kong"}, ignore_index=True)
countries_new.to_csv("/Users/ameliemeurer/Documents/Amélie/HEC M2/07. Research Paper - UN Women/02. Code/domestic_violence_covid-19/data/countries.csv")
                                  """


# Aggregation for Graph 1. Scatter-Testimonial


In [ ]:
df_graph_1 = df_all[["submitted_at","testimonial"]]

In [ ]:
# add here the decision whether or not to publish it. for now, display if there is a testimonial

display_testimonial = [1 if testimonial != 0 else 0 for testimonial in df_graph_1["testimonial"]]

In [ ]:
df_graph_1["display_testimonial"] = display_testimonial

In [ ]:
df_graph_1

In [ ]:
df_graph_1.to_csv("data_graph1.csv")